# US Employment Heatmap

This notebook takes Census data on employment by state in 2015, then heatmaps it onto a US map.
It uses geopandas, as well as pandas and matplotlib.
In order to run, first pip install geopandas, then the descartes package.

In [1]:
import pandas as pd
import numpy as np
import requests
import plotly
plotly.tools.set_credentials_file(username='nnabil', api_key='WnvQXaScR0wSes6mVrnF')
import plotly.plotly as py
import plotly.io as pio
from matplotlib import pyplot as plt

# note: will need to pip install geopandas and descartes

# define path for source csv file
source_file_2015 = 'Resources/employment/ACS_15_5YR_S2301_with_ann.csv'


### Read and clean employment data.

In [2]:
# read csv file into dataframe
data_2015 = pd.read_csv(source_file_2015)
data_2015.head(3)

,GEO.id,State Code,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,...,HC02_EST_VC43,HC02_MOE_VC43,HC03_EST_VC43,HC03_MOE_VC43,HC04_EST_VC43,HC04_MOE_VC43,HC01_EST_VC44,HC01_MOE_VC44,HC02_EST_VC44,HC02_MOE_VC44
0,Id,Code,Geography,Total; Estimate; Population 16 years and over,Total; Margin of Error; Population 16 years an...,Labor Force Participation Rate; Estimate; Popu...,Labor Force Participation Rate; Margin of Erro...,Employment/Population Ratio; Estimate; Populat...,Employment/Population Ratio; Margin of Error; ...,Unemployment rate; Estimate; Population 16 yea...,...,Labor Force Participation Rate; Estimate; EDUC...,Labor Force Participation Rate; Margin of Erro...,Employment/Population Ratio; Estimate; EDUCATI...,Employment/Population Ratio; Margin of Error; ...,Unemployment rate; Estimate; EDUCATIONAL ATTAI...,Unemployment rate; Margin of Error; EDUCATIONA...,Total; Estimate; EDUCATIONAL ATTAINMENT - Popu...,Total; Margin of Error; EDUCATIONAL ATTAINMENT...,Labor Force Participation Rate; Estimate; EDUC...,Labor Force Participation Rate; Margin of Erro...
1,0400000US01,AL,Alabama,3846845,1679,58.3,0.2,52.6,0.2,9.3,...,71.5,0.2,65.8,0.2,7.5,0.1,344267,4958,50.1,0.7
2,0400000US02,AK,Alaska,565212,516,70.6,0.4,62.1,0.4,8.2,...,79.4,0.4,71.6,0.5,6.9,0.3,26191,1143,59.8,1.7


In [3]:
# choose the three columns to keep
data_2015 = data_2015[['State Code', 'GEO.display-label','HC02_EST_VC01']]

# drop gratuitious header line
data_2015 = data_2015.drop(data_2015.index[0])

# rename columns 
data_2015.columns = ['Code', 'State', 'Employment']

# show sample of final dataframe
data_2015.head(2)


,Code,State,Employment
1,AL,Alabama,58.3
2,AK,Alaska,70.6


### Map data

In [4]:
#For loop

for col in data_2015.columns:
    data_2015[col] = data_2015[col].astype(str)
    
       
# prepare data to put into text
# text variable will be inserted into data dictionary below
# will be used for hover functionality
    
data_2015['Text'] = data_2015['State'] + 'has ' + data_2015['Employment'] + ' employment'
       
# set up data dictionary
data = [ dict(
        type='choropleth',
        colorscale = 'Greens',
        reversescale = True,
        autocolorscale = False,
        locations = data_2015['Code'],
        z = data_2015['Employment'].astype(float),
        locationmode = 'USA-states',
        text = data_2015['Text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Employment Rates"
        )
    ) ]
       
# Set up layout dictionary for the visualization
layout = dict(
        title = 'Employment Rate by Participating State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

# Name the figure as a dictionary of the 1) data dictionary and 2) layout dictonary 
fig = dict( data=data, layout=layout )

#save fig
pio.write_image(fig, 'Images/employment.png')

# Name the file for your inline viewing 
# url = py.iplot( fig, filename='d3-cloropleth-map' )
py.iplot(data, filename='d3-cloropleth-map')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~nnabil/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'
